In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression


In [4]:
# Hochladen beider .csv Dateien: kunden & besucher
kunden = pd.read_csv('kunden.csv', sep = ';')
besucher = pd.read_csv('besucher.csv', sep = ';')

# Hinzufügen einer neuen Spalte mit einem Standardwert
besucher['Preis'] = 'NaN'

# Anpassen der Spaltenreihenfolge, um 'Preis'-Spalte in besucher als dritte Spalte zu setzen
cols = list(besucher.columns)
cols.insert(2, cols.pop(cols.index('Preis')))
besucher = besucher[cols]

# Zusammenfügen der DataFrames untereinander (vertikal)
combined_draft = pd.concat([kunden, besucher], ignore_index=True, axis = 0)

# Sortieren des kombinierten DataFrames nach der Spalte 'KundeNr' aufsteigend
combined_sorted = combined_draft.sort_values(by='KundeNr')

# Gruppieren der Werte nach Überschrift
combined_gruppen = combined_draft.groupby('Geschlecht').size().reset_index(name='Anzahl')

# Zählen der Fehlerwerte in allen Spalten
na_count = combined_draft.isna().sum()

# Ausgabe des sortierten DataFrames
print(combined_draft)

      Alter          Einkommen     Preis  Geschlecht  Zeit KundeNr
0        64            66894.0  88160.31         1.0  43.0   K0310
1        54            77644.0  103145.7         1.0  40.0   K1042
2        55            44341.0  80565.16         0.0  37.0   K0382
3        49            67271.0  83949.89         0.0  42.0   K0498
4        46            49832.0  93781.58         0.0  41.0   K0552
...     ...                ...       ...         ...   ...     ...
1990     32  34391,82496545645       NaN         1.0  48,0   K1211
1991     41  68516,49543941053       NaN         0.0  34,0   K1375
1992     65  89685,33344089528       NaN         0.0  23,0   K1965
1993     68  95714,30526812703       NaN         0.0  32,0   K1540
1994     49   73319,1870845747       NaN         0.0  27,0   K1207

[1995 rows x 6 columns]


In [6]:
land = pd.read_csv('geo.txt', sep = '\t')

# Ersetzen der Werte in der Spalte "Niederlassung"
land['Niederlassung'] = land['Niederlassung'].replace({
    'NRW': 'Nordrhein-Westfalen',
    'Düsseldorf': 'Nordrhein-Westfalen',
    'Berlin-Charlottenburg': 'Berlin',
    'Berlin-Mitte': 'Berlin',
    'BERLIN': 'Berlin'
})

# Gruppieren nach der Spalte "Niederlassung" und Zählen der Einträge in jeder Gruppe/ Überprüfen auf Richtigkeit der Bundesländer
land_gruppen = land.groupby('Niederlassung').size().reset_index(name='Anzahl')

land

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,Nordrhein-Westfalen
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern
...,...,...
1990,K1991,Hessen
1991,K1992,Baden-Württemberg
1992,K1993,Nordrhein-Westfalen
1993,K1994,Berlin


In [7]:
# Schritt 4: Anpassen der Spaltenreihenfolge, um "Niederlassung" als siebente (in Python sechste, weil Beginn = 0) Spalte zu setzen
cols = list(combined_draft.columns)
cols.insert(6, cols.pop(cols.index('KundeNr')))
combined = combined_draft[cols]

# Zusammenführen von "combined_draft" und "land", Sortierung nach "KundeNr" und Einfügen der "Niederlassung" Werte in Spalte "Niederlassung"
combined = pd.merge(combined_draft, land, on='KundeNr', how='right')

# Ersetzen von Kommas durch Punkte in allen Spalten
combined['Zeit'] = combined['Zeit'].astype(str).str.replace(',', '.', regex=False)
combined['Einkommen'] = combined['Einkommen'].astype(str).str.replace(',', '.', regex=False)
combined['Preis'] = combined['Preis'].astype(str).str.replace(',', '.', regex=False)
combined['Geschlecht'] = combined['Geschlecht'].astype(str).str.replace(',', '.', regex=False)

# Schritt 5: Speichern des aktualisierten DataFrames in eine neue CSV-Datei
#combined_df.to_csv('final_project/kunden_mit_niederlassung.csv', index=False)

combined

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Niederlassung
0,68,89437.0,114481.9,1.0,62.0,K0001,Sachsen
1,58,65271.0,106258.99,1.0,44.0,K0002,Nordrhein-Westfalen
2,44,48195.0,85705.57,1.0,29.0,K0003,Hessen
3,37,48957.0,76412.36,0.0,36.0,K0004,Bayern
4,50,56621.0,92898.67,0.0,40.0,K0005,Bayern
...,...,...,...,...,...,...,...
1990,48,59967.31328247114,NaN,1.0,21.0,K1991,Hessen
1991,67,81521.76144297997,NaN,1.0,21.0,K1992,Baden-Württemberg
1992,40,41616.57298010873,NaN,0.0,48.0,K1993,Nordrhein-Westfalen
1993,49,76602.7888395788,NaN,1.0,19.0,K1994,Berlin


In [15]:
# Speichern des aktualisierten DataFrames in eine neue CSV-Datei
combined.to_csv('combined.csv', index=False)

# F2. Was ist der Höchst-, Mindest- und Durchschnittspreis der verkauften Autos?


In [10]:
max_preis = combined['Preis'].astype(float).max()
min_preis = combined['Preis'].astype(float).min()
avg_preis = combined['Preis'].astype(float).mean()

print(f"F2: Höchstpreis: {max_preis}, Mindestpreis: {min_preis}, Durchschnittspreis: {avg_preis}")

F2: Höchstpreis: 165482.31, Mindestpreis: 21471.65, Durchschnittspreis: 88975.19871376811


# F5. Wie hoch war der durchschnittliche Umsatz pro Bundesland?

In [11]:
bundeslaender= combined[combined['Preis'].notna()]['Niederlassung'].unique()

umsatz_pro_bundesland = {}

for bundesland in bundeslaender:
    umsatz_pro_bundesland[bundesland] = combined[(combined['Niederlassung'] == bundesland) & (combined['Preis'].notna())]['Preis'].astype(float).mean()
    
umsatz_pro_bundesland_df = pd.DataFrame(umsatz_pro_bundesland.items(), columns=['Niederlassung','Durchschnittlicher Umsatz'])

umsatz_pro_bundesland_df['Durchschnittlicher Umsatz'] = umsatz_pro_bundesland_df['Durchschnittlicher Umsatz'].round(2)
umsatz_pro_bundesland_df = umsatz_pro_bundesland_df.sort_values(by='Durchschnittlicher Umsatz', ascending=False)


print(f"F5: Durchschnittlicher Umsatz pro Bundesland:\n{umsatz_pro_bundesland_df.to_string(index=False)}")


F5: Durchschnittlicher Umsatz pro Bundesland:
      Niederlassung  Durchschnittlicher Umsatz
  Baden-Württemberg                   90969.56
          Thüringen                   90816.99
             Berlin                   89584.98
            Sachsen                   89455.08
             Bayern                   89344.31
Nordrhein-Westfalen                   88633.27
      Niedersachsen                   87963.31
        Brandenburg                   86673.64
             Hessen                   85864.93
            Hamburg                   85549.50


# F8. Wie hoch ist das Durchschnittsalter unserer Kunden? Wie hoch ist das Durchschnittsalter unserer Besucher?


In [12]:
#Funktion zur Entfernung von Ausreißern IQR:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column]>= lower_bound)&(df[column]<= upper_bound)]


In [16]:
#Aufteilung Kunden/Besucher
kunden = combined[combined['Preis'].notna()]
besucher = combined[combined['Preis'].isna()]

#Aureißer werden entfernt
kunden = remove_outliers(kunden, 'Alter')
besucher = remove_outliers(besucher, 'Alter')

#Durchschnittsalter der beiden Gruppen
avg_alter_kunden = kunden['Alter'].astype(float).mean()
avg_alter_besucher = besucher['Alter'].astype(float).mean()

#Print
print(f"F8: Durchschnittliches Alter der Kunden: {avg_alter_kunden}")
print(f"F8: Durchschnittliches Alter der Besucher: {avg_alter_besucher}")

#HIER STIMMT IRGENDWAS NICHT BEI BESUCHER!


F8: Durchschnittliches Alter der Kunden: 50.153537380832915
F8: Durchschnittliches Alter der Besucher: nan
